# 2018年度　パターン認識最終課題

パターン認識最終課題では、Titanicの乗客に関するデータセットを使って、乗客の等級(Pclass)を予測するモデルを構築してもらいます。

## 課題１: 色々なモデルを試してみよう！
まずは、講義で扱った様々な識別機を試してみましょう。
以下に示すロジスティック回帰の例を参考に、指示に従いSVM、K-近傍法、決定木、ランダムフォレストを実装を行ってもらいます。

In [1]:
from IPython.display import display
import pandas as pd
from pandas import DataFrame, Series
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

### Step1 データの読み込み
まずは、データを読み込みます。

In [2]:
df_train_data = pd.read_csv('train.csv')

In [3]:
df_train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


今回は、この中で、Pclassを予測するモデルの構築を行ってもらいます。

### Step 2　使用する列の抽出
次に、使用する列だけを取り出します。　今回はPclassと、予測に使えそうなSex, Age, SibSp, Parch, Fareを取り出します。

In [4]:
y_ = df_train_data.Pclass # ラベルデータ

In [5]:
DataFrame(y_).head(10)

,Pclass
0,3
1,1
2,3
3,1
4,3
5,3
6,1
7,3
8,3
9,2


ラベルデータの上位10件を表示しました。

In [6]:
X_ = df_train_data[['Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

In [7]:
X_.tail(10)

,Sex,Age,SibSp,Parch,Fare
590,male,35.0,0,0,7.1250
591,female,52.0,1,0,78.2667
592,male,47.0,0,0,7.2500
593,female,NaN,0,2,7.7500
594,male,37.0,1,0,26.0000
595,male,36.0,1,1,24.1500
596,female,NaN,0,0,33.0000
597,male,49.0,0,0,0.0000
598,male,NaN,0,0,7.2250
599,male,49.0,1,0,56.9292


特徴量データの上位10件を表示しました。

### Step3　特徴量データの前処理
識別機での学習を行う前に、特徴量を適切な形へ変換してあげる必要があります。
その過程を見ていきましょう。

まず、欠損値を含むデータの扱いですが、本課題では、簡単のため単純に削除する手法を取ります。

In [8]:
df_temp = pd.concat([y_, X_], axis=1).dropna(how='any')# 欠損値を含むデータの削除
df_temp.index = range(len(df_temp)) 
y = df_temp.Pclass
X = df_temp[['Sex', 'Age', 'SibSp', 'Parch', 'Fare']]

次にSex列は文字列であるため、数値に変換してあげる必要があります。ここでは、male=0, female=1として変換します。

In [9]:
X.tail()

,Sex,Age,SibSp,Parch,Fare
469,male,47.0,0,0,7.2500
470,male,37.0,1,0,26.0000
471,male,36.0,1,1,24.1500
472,male,49.0,0,0,0.0000
473,male,49.0,1,0,56.9292


In [10]:
sex_le = LabelEncoder()
X.Sex = sex_le.fit_transform(X.Sex)

In [11]:
X.tail(10)

,Sex,Age,SibSp,Parch,Fare
464,1,47.0,0,0,15.0000
465,1,60.0,1,1,79.2000
466,1,22.0,0,0,8.0500
467,1,35.0,0,0,7.1250
468,0,52.0,1,0,78.2667
469,1,47.0,0,0,7.2500
470,1,37.0,1,0,26.0000
471,1,36.0,1,1,24.1500
472,1,49.0,0,0,0.0000
473,1,49.0,1,0,56.9292


これで、数値データに置き換えられました。


最後に、それぞれの特徴量の数値の変動量に差による影響力の差を排除するため、正規化を行います。

In [12]:
mm_scaler = MinMaxScaler()

In [13]:
X.Age = DataFrame(mm_scaler.fit_transform(DataFrame(X.Age.copy())))
X.SibSp = DataFrame(mm_scaler.fit_transform(DataFrame(X.SibSp.copy())))
X.Parch = DataFrame(mm_scaler.fit_transform(DataFrame(X.Parch.copy())))
X.Fare = DataFrame(mm_scaler.fit_transform(DataFrame(X.Fare.copy())))

In [14]:
X.head(10)

,Sex,Age,SibSp,Parch,Fare
0,1,0.302491,0.2,0.0,0.014151
1,0,0.530249,0.2,0.0,0.139136
2,0,0.359431,0.0,0.0,0.015469
3,0,0.487544,0.2,0.0,0.103644
4,1,0.487544,0.0,0.0,0.015713
5,1,0.758007,0.0,0.0,0.101229
6,1,0.017794,0.6,0.2,0.041136
7,0,0.373665,0.0,0.4,0.021731
8,0,0.188612,0.2,0.0,0.058694
9,0,0.046263,0.2,0.2,0.032596


これで、正規化が出来ました。

### Step 4　識別器の実装
ロジスティック回帰の例を参考に、SVM、K-近傍法、決定木、ランダムフォレストを実装し、その考察を行いましょう。<br><br>
精度をできるだけ正確に求めるために，それぞれの手法について１０回繰り返す。

#### ロジスティック回帰
ロジスティック回帰の例を示します。実装にはscikit-laern(http://scikit-learn.org/stable/) を使用します。

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 訓練データとテストデータに分割

In [16]:
logistic_regression = LogisticRegression() # ロジスティック回帰モデルの作成
total = 0
for i in range(10):
    logistic_regression.fit(X_train, y_train) # 訓練データを使って学習
    y_pred = logistic_regression.predict(X_test) # 予測値の取得
    total += accuracy_score(y_pred=y_pred, y_true=y_test) #合計 
average_lr = total/10 #平均
    
print('accuracy:{}'.format(average_lr)) # 精度の計算

accuracy:0.6458333333333333


#### SVM
ここで、SVMの実装を行ってください。特徴量も自由に選択して構いません。

In [17]:
svm = SVC() #SVMモデルの作成
total = 0
for i in range(10):
    svm.fit(X_train, y_train) #訓練データを使って学習
    y_pred = svm.predict(X_test) #予測値の取得
    total += accuracy_score(y_pred=y_pred, y_true=y_test) #合計 
average_svm = total/10 #平均

print('accuracy:{}'.format(average_svm )) # 精度の計算

accuracy:0.625


#### K-近傍法

ここで、K-近傍法の実装を行ってください。特徴量も自由に選択して構いません。

In [18]:
knn = KNeighborsClassifier() #K-近傍法モデルの作成
total = 0
for i in range(10):
    knn.fit(X_train, y_train) #訓練データを使って学習
    y_pred = knn.predict(X_test) #予測値の取得
    total += accuracy_score(y_pred=y_pred, y_true=y_test) #合計 
average_knn = total/10 #平均

print('accuracy:{}'.format(average_knn)) # 精度の計算

accuracy:0.75


#### 決定木

ここで、決定木の実装を行ってください。特徴量も自由に選択して構いません。

In [19]:
decision_tree = DecisionTreeClassifier() #決定木モデルの作成
total = 0
for i in range(10):
    decision_tree.fit(X_train, y_train) #訓練データを使って学習
    y_pred = decision_tree.predict(X_test) #予測値の取得
    total += accuracy_score(y_pred=y_pred, y_true=y_test) #合計 
average_dt = total/10 #平均

print('accuracy:{}'.format(average_dt)) # 精度の計算

accuracy:0.8958333333333334


#### ランダムフォレスト

ここで、ランダムフォレストの実装を行ってください。特徴量も自由に選択して構いません。

In [20]:
random_forest = RandomForestClassifier() #ランダムフォレストモデルの作成
total = 0
for i in range(10):
    random_forest.fit(X_train, y_train) #訓練データを使って学習
    y_pred = random_forest.predict(X_test) #予測値の取得
    total += accuracy_score(y_pred=y_pred, y_true=y_test) #合計 
average_rf = total/10 #平均

print('accuracy:{}'.format(average_rf)) # 精度の計算

accuracy:0.9


## 課題２: モデルの改善を行おう！
課題1で作成した各モデルについて、どうすればさらに良い精度を出すことができるのか、パラメータの調整や特徴量の選定などを行いましょう。
最後に、この課題において最善と考えるモデルを述べ、その理由も示してください。

まず，精度を高めるために，新たなデータ（変数）の追加と特徴量の選定を行う．

（１）sexのmaleを0に，femaleを1に置き換え，EmbarkedのSを0，Cを1，Qを2に置き換える． 

In [21]:
# SexとEmbarkedを数値に変換
train = df_train_data.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

（２）Cabinは無視し，AgeとEmbarkedの欠損値を平均値で埋める．

In [22]:
# AgeとEmbarkedの欠損値を平均で補完
train["Age"].fillna(train.Age.mean(), inplace=True) 
train["Embarked"].fillna(train.Embarked.mean(), inplace=True)

（３）Nameを敬称を基に数値化する．

In [23]:
# Nameを敬称を基に数値化
combine1 = [train]

for train in combine1: 
    train['Salutation'] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 
#     print(train['Salutation'])
for train in combine1: 
    train['Salutation'] = train['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    train['Salutation'] = train['Salutation'].replace('Mlle', 'Miss')
    train['Salutation'] = train['Salutation'].replace('Ms', 'Miss')
    train['Salutation'] = train['Salutation'].replace('Mme', 'Mrs')
    del train['Name']

# "Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 
for train in combine1: 
    train['Salutation'] = train['Salutation'].map(Salutation_mapping) 
    train['Salutation'] = train['Salutation'].fillna(0)

（４）Ticketを先頭の文字と文字数を基に数値化する．

In [24]:
# Ticketを頭文字と文字数を基に数値化
for train in combine1: 
    train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
    train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x)) 
    train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'], np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')) 
    train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
    del train['Ticket'] 
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)

（５）CabinもTicketと同様に先頭の文字と文字数を基に数値化する．

In [25]:
# Cabinを頭文字と文字数を基に数値化
for train in combine1: 
    train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
    train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
    train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))

del train['Cabin'] 
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)

（６）家族の人数（FamilySize）という新たな変数を導入する．Sibspは乗っていた夫婦と兄弟の人数を，Parchは乗っていた親と子供の人数を表したものであるので，Sibsp+Parch+1がFamilySizeとなる．また，FamilySizeが1のとき，IsAlone（一人で乗っているかどうか）が1となる．

In [26]:
#新たな変数FamilySizeを導入
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1

for test in combine1:
    train['IsAlone'] = 0
    train.loc[test['FamilySize'] == 1, 'IsAlone'] = 1

In [27]:
train.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett,FamilySize,IsAlone
0,1,0,3,0,22.0000,1,0,7.2500,0.0,1.0,3,9,0,2,0
1,2,1,1,1,38.0000,1,0,71.2833,1.0,3.0,0,8,1,2,0
2,3,1,3,1,26.0000,0,0,7.9250,0.0,2.0,3,16,0,1,1
3,4,1,1,1,35.0000,1,0,53.1000,0.0,3.0,1,6,1,2,0
4,5,0,3,0,35.0000,0,0,8.0500,0.0,1.0,3,6,0,1,1
5,6,0,3,0,29.7827,0,0,8.4583,2.0,1.0,3,6,0,1,1
6,7,0,1,0,54.0000,0,0,51.8625,0.0,1.0,1,5,2,1,1
7,8,0,3,0,2.0000,3,1,21.0750,0.0,4.0,3,6,0,5,0
8,9,1,3,1,27.0000,0,2,11.1333,0.0,3.0,3,6,0,3,0
9,10,1,2,1,14.0000,1,0,30.0708,1.0,3.0,2,6,0,2,0


（７）相関行列を作り，Pclassの相関係数を取り出す．

In [28]:
# 相関行列の作成
co_matrix = pd.DataFrame(np.corrcoef(train.T), index=train.T.index, columns=train.columns)
# Pclassの相関係数を取り出す
co_matrix_Pclass = co_matrix['Pclass']
print(co_matrix_Pclass)

PassengerId   -0.085506
Survived      -0.293376
Pclass         1.000000
Sex           -0.122969
Age           -0.331857
SibSp          0.091819
Parch         -0.008717
Fare          -0.593298
Embarked       0.025239
Salutation    -0.115327
Ticket_Lett    0.616742
Ticket_Len     0.065076
Cabin_Lett    -0.697704
FamilySize     0.059253
IsAlone        0.142660
Name: Pclass, dtype: float64


（８）Pclassの相関係数のうち絶対値が0.2より大きいものを特徴量として選定する．

In [142]:
# Pclassの相関係数のうち絶対値が0.2より小さいものを取り除き，特徴量を選定
label_drop = []

for label in co_matrix_Pclass.index:
    if abs(co_matrix_Pclass[label]) < 0.2:
        label_drop.append(label)

print('###Dropped columns###')
print(label_drop)
new_train = train.drop(label_drop, axis=1)

###Dropped columns###
['PassengerId', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Salutation', 'Ticket_Len', 'FamilySize', 'IsAlone']


（９）特徴量を選定したnew_trainをPclassとそれ以外に分け，正規化をする．

In [143]:
X2 = new_train.drop('Pclass', axis=1)
y2 = new_train['Pclass']

# 正規化
mm_scaler = MinMaxScaler()

for label in X2.columns:
    X2[label] = DataFrame(mm_scaler.fit_transform(DataFrame(X2[label].copy())))

In [144]:
X2.head(10)

,Survived,Age,Fare,Ticket_Lett,Cabin_Lett
0,0.0,0.302491,0.014151,1.000000,0.0
1,1.0,0.530249,0.139136,0.000000,0.5
2,1.0,0.359431,0.015469,1.000000,0.0
3,1.0,0.487544,0.103644,0.333333,0.5
4,0.0,0.487544,0.015713,1.000000,0.0
5,0.0,0.413277,0.016510,1.000000,0.0
6,0.0,0.758007,0.101229,0.333333,1.0
7,0.0,0.017794,0.041136,1.000000,0.0
8,1.0,0.373665,0.021731,1.000000,0.0
9,1.0,0.188612,0.058694,0.666667,0.0


（１０）X2とy2を訓練データとテストデータに分割する．

In [130]:
# 訓練データとテストデータに分割
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.1) 

<h3>###精度比較###</h3><br>
ここで，課題１と課題２の精度を比較するために，パラメータと特徴量データの組み合わせを次の4つで考える．<br><br>
1-1：パラメータ（課題１）　特徴量データ（課題１）<br>
1-2：パラメータ（課題１）　特徴量データ（課題２）<br>
2-1：パラメータ（課題２）　特徴量データ（課題１）<br>
2-2：パラメータ（課題２）　特徴量データ（課題２）<br>


各手法において１０回繰り返し，平均を課題２の精度とする．<br>
ただし，SVMとK-近傍法はアルゴリズムを考えると各回の精度は同じであるため，１回だけ行うことにする．

#### SVM
ここで、SVMの実装を行ってください。パラメータ、特徴量を工夫しましょう。

In [131]:
#１−１（パラメータ：課題１，　特徴量データ：課題１）
svm1 = SVC() #SVMモデルの作成
svm1.fit(X_train, y_train) #訓練データを使って学習
y_pred_svm1_1 = svm1.predict(X_test) #予測値の取得
print('###1-1### accuracy:{}'.format(accuracy_score(y_pred=y_pred_svm1_1, y_true=y_test))) # 1-1の精度の計算


#１−２（パラメータ：課題１，　特徴量データ：課題２）
svm1.fit(X2_train, y2_train)
y_pred_svm1_2 = svm1.predict(X2_test)
print('###1-2### accuracy:{}'.format(accuracy_score(y_pred=y_pred_svm1_2, y_true=y2_test))) # 1-2の精度の計算
    

    
#最適化したいパラメータをリストで定義（C：ソフトマージンの強さ, kernel：カーネルの種類, gamma：カーネル係数）
new_parameters = {'C':[1,10, 100, 1000], 'kernel':['linear', 'rbf'], 'gamma':[0.01, 0.1, 1]}

#グリッドサーチにより最適値を求める
svm2 = GridSearchCV(SVC(), new_parameters, cv=5, scoring='accuracy', n_jobs = -1)

#２−１（パラメータ：課題２，　特徴量データ：課題１）
svm2.fit(X_train, y_train)
y_pred_svm2_1 = svm2.predict(X_test)
print('###2-1### accuracy:{}'.format(accuracy_score(y_pred=y_pred_svm2_1, y_true=y_test))) # 2-1の精度の計算

#２−２（パラメータ：課題２，　特徴量データ：課題２）
svm2.fit(X2_train, y2_train)
y_pred_svm2_2 = svm2.predict(X2_test)
print('###2-2### accuracy:{}'.format(accuracy_score(y_pred=y_pred_svm2_2, y_true=y2_test))) # 2-2の精度の計算

###1-1### accuracy:0.625
###1-2### accuracy:0.8
###2-1### accuracy:0.8333333333333334
###2-2### accuracy:0.8833333333333333


#### K-近傍法

ここで、K-近傍法の実装を行ってください。パラメータ、特徴量を工夫しましょう。

In [132]:
#１−１（パラメータ：課題１，　特徴量データ：課題１）
knn1 = KNeighborsClassifier() #K-近傍法モデルの作成
knn1.fit(X_train, y_train) #訓練データを使って学習
y_pred_knn1_1 = knn1.predict(X_test) #予測値の取得
print('###1-1### accuracy:{}'.format(accuracy_score(y_pred=y_pred_knn1_1, y_true=y_test))) # 1-1の精度の計算

#１−２（パラメータ：課題１，　特徴量データ：課題２）
knn1.fit(X2_train, y2_train)
y_pred_knn1_2 = knn1.predict(X2_test)
print('###1-2### accuracy:{}'.format(accuracy_score(y_pred=y_pred_knn1_2, y_true=y2_test))) # 1-2の精度の計算


#最適化したいパラメータをリストで定義（n_neighbors：kの値, weights : 重み付け）
new_parameters = {'n_neighbors':[2, 4, 6, 8, 10], 'weights':['uniform', 'distance']}

#グリッドサーチにより最適値を求める
knn2 = GridSearchCV(KNeighborsClassifier(), new_parameters, cv=5, scoring='accuracy', n_jobs=-1)

#２−１（パラメータ：課題２，　特徴量データ：課題１）
knn2.fit(X_train, y_train)
y_pred_knn2_1 = knn2.predict(X_test)
print('###2-1### accuracy:{}'.format(accuracy_score(y_pred=y_pred_knn2_1, y_true=y_test))) # 2-1の精度の計算

#２−２（パラメータ：課題２，　特徴量データ：課題２）
knn2.fit(X2_train, y2_train)
y_pred_knn2_2 = knn2.predict(X2_test)
print('###2-2### accuracy:{}'.format(accuracy_score(y_pred=y_pred_knn2_2, y_true=y2_test))) # 2-2の精度の計算

###1-1### accuracy:0.75
###1-2### accuracy:0.85
###2-1### accuracy:0.8541666666666666
###2-2### accuracy:0.9


#### 決定木

ここで、決定木の実装を行ってください。パラメータ、特徴量を工夫しましょう。

In [139]:
#１−１（パラメータ：課題１，　特徴量データ：課題１）
decision_tree1 = DecisionTreeClassifier() #決定木モデルの作成
total = 0
for i in range(10):
    decision_tree1.fit(X_train, y_train) #訓練データを使って学習
    y_pred_dt1_1 = decision_tree1.predict(X_test) #予測値の取得
    total += accuracy_score(y_pred=y_pred_dt1_1, y_true=y_test)
average_dt1_1 = total/10 
print('###1-1### accuracy:{}'.format(average_dt1_1)) # 1-1の精度の計算

#１−２（パラメータ：課題１，　特徴量データ：課題２）
total = 0
for i in range(10):
    decision_tree1.fit(X2_train, y2_train)
    y_pred_dt1_2 = decision_tree1.predict(X2_test)
    total += accuracy_score(y_pred=y_pred_dt1_2, y_true=y2_test)
average_dt1_2 = total/10 
print('###1-2### accuracy:{}'.format(average_dt1_2)) # 1-2の精度の計算


#最適化したいパラメータをリストで定義（max_depth：作成する決定木の深さの最大値）
new_parameters  = {"max_depth":[1, 5, 10, 15]}

#グリッドサーチにより最適値を求める
decision_tree2 = GridSearchCV(DecisionTreeClassifier(), new_parameters, cv=5, scoring='accuracy', n_jobs=-1)

#２−１（パラメータ：課題２，　特徴量データ：課題１）
total = 0
for i in range(10):
    decision_tree2.fit(X_train, y_train)
    y_pred_dt2_1 = decision_tree2.predict(X_test)
    total += accuracy_score(y_pred=y_pred_dt2_1, y_true=y_test)
average_dt2_1 = total/10 
print('###2-1### accuracy:{}'.format(average_dt2_1)) # 2-1の精度の計算


#２−２（パラメータ：課題２，　特徴量データ：課題２）
total = 0
for i in range(10):
    decision_tree2.fit(X2_train, y2_train)
    y_pred_dt2_2 = decision_tree2.predict(X2_test)
    total += accuracy_score(y_pred=y_pred_dt2_2, y_true=y2_test)
average_dt2_2 = total/10 
print('###2-2### accuracy:{}'.format(average_dt2_2)) # 2-2の精度の計算

###1-1### accuracy:0.8958333333333334
###1-2### accuracy:0.9650000000000001
###2-1### accuracy:0.8958333333333334
###2-2### accuracy:0.9716666666666665


#### ランダムフォレスト

ここで、ランダムフォレストの実装を行ってください。パラメータ、特徴量を工夫しましょう。

In [141]:
#１−１（パラメータ：課題１，　特徴量データ：課題１）
random_forest1 = RandomForestClassifier() #ランダムフォレストモデルの作成
total = 0
for i in range(10):
    random_forest1.fit(X_train, y_train) #訓練データを使って学習
    y_pred_rf1_1 = random_forest1.predict(X_test) #予測値の取得
    total += accuracy_score(y_pred=y_pred_rf1_1, y_true=y_test)
average_rf1_1 = total/10 
print('###1-1### accuracy:{}'.format(average_rf1_1)) # 1-1の精度の計算

#１−２（パラメータ：課題１，　特徴量データ：課題２）
total = 0
for i in range(10):
    random_forest1.fit(X2_train, y2_train)
    y_pred_rf1_2 = random_forest1.predict(X2_test)
    total += accuracy_score(y_pred=y_pred_rf1_2, y_true=y2_test)
average_rf1_2 = total/10 
print('###1-2### accuracy:{}'.format(average_rf1_2)) # 1-2の精度の計算


#最適化したいパラメータをリストで定義（n_estimators：木の数を設定するためのパラメータ，　max_depth：作成する決定木の深さの最大値）
new_parameters = {'n_estimators':[100], "max_depth":[20]}

#グリッドサーチにより最適値を求める
random_forest2 = GridSearchCV(RandomForestClassifier(), new_parameters, cv=5, scoring='accuracy', n_jobs=-1)
#２−１（パラメータ：課題２，　特徴量データ：課題１）
total = 0
for i in range(10):
    random_forest2.fit(X_train, y_train)
    y_pred_rf2_1 = random_forest2.predict(X_test)
    total += accuracy_score(y_pred=y_pred_rf2_1, y_true=y_test)
average_rf2_1 = total/10 
print('###2-1### accuracy:{}'.format(average_rf2_1)) # 2-1の精度の計算

#２−２（パラメータ：課題２，　特徴量データ：課題２）
total = 0
for i in range(10):
    random_forest2.fit(X2_train, y2_train)
    y_pred_rf2_2 = random_forest2.predict(X2_test)
    total += accuracy_score(y_pred=y_pred_rf2_2, y_true=y2_test)
average_rf2_2 = total/10 
print('###2-2### accuracy:{}'.format(average_rf2_2)) # 2-2の精度の計算

###1-1### accuracy:0.8895833333333334
###1-2### accuracy:0.9816666666666667
###2-1### accuracy:0.9041666666666668
###2-2### accuracy:0.985


最後に、以下のコードの指定された部分を書き換え、実行して下さい。

In [107]:
best_model = random_forest#(ここに最善と考えるモデルを指定)　課題1のロジスティック回帰で、logistic_regression にあたるもの
best_X_train = X2_train# (ここに上記モデルの学習時に使用した特徴量データを指定)
best_y_train = y2_train# (ここに上記モデルの学習時に使用したラベルテータを指定)

pickle.dump(best_model, open('best_model.pkl','wb'))
best_X_train.to_csv('best_X_train.csv')
best_y_train.to_csv('best_y_train.csv')


### 考察
この課題において最善と考えるモデルを述べ、その理由も示してください。

<h5>★最善と考えられるモデル<br></h5>
ランダムフォレスト<br>
<h5>★理由</h5><br>
それぞれの手法について今回の学習においてなぜランダムフォレストに劣ったのかを考えてみる．<br><br>
＜ロジステック回帰＞<br>
ロジスティック回帰は，もともと２クラス分類のための回帰分析であるので，多クラス分類も行うことは可能であるが，複雑になり，精度はあまり高くならない．
<br>＜SVM＞<br>
SVMは，データの特徴の次元数が大きくても識別精度は良く，最適化するパラメータも比較的少ないため，パラメータの算出が簡易である．しかし，SVMも基本的には２クラス分類にしか使えないため，多クラス分類に用いると精度はあまり高くはならない．また，次元の呪いの影響も非常に受けやすい．
<br>＜K-近傍法＞<br>
K-近傍法は，局所的な推測に有効な手法で，全データを使わないので，外れ値の影響を受けにくい．一方，パラメータKの決め方によって，結果が大きく変わるため，妥当性の評価が困難である．また，狭い領域で距離が短い場合は，推測値の妥当性は良いが，距離が長くなってしまうと，妥当性が悪くなり，精度は低くなる．さらに，モデルを作らないため，特徴量がクラスとどのような関連があるのかを理解することが難しいため，精度を上げることは難しい．
<br>＜決定木＞<br>
決定木は，処理結果が解釈しやすく妥当性を判断しやすい．一方，データの前処理に伴う負担が少なく，モデル作成時に，利用したデータに対して過剰に適合してしまい，分割回数を増やすほど汎化性能が低くなる．
<br>＜ランダムフォレスト<br>
ランダムフォレストは，並列処理が簡単で，重要な特徴量を知ることができ,アンサンブル学習のため，過学習を起こしにくい．一方，学習データや特徴量データをランダムに抽出するので，データと変数が少なすぎると，うまく学習できない．複雑なデータでは，SVMに比べると汎化性能が劣る．<br>


これらを踏まえて，ランダムフォレストの精度がよかった理由をまとめる.<br>
・今回の学習では，十分なデータと変数があったこと<br>
・データがあまり複雑ではなかったこと<br>
・特徴量の選定がうまくできたこと<br>
・相関係数を考えることで，Pclassとの関係性が大きいものだけを用いて，ノイズを減らすことができたこと<br>
